In [ ]:
import feedparser
import dateutil.parser
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment
import mongo_utils
from datetime import datetime as dt

In [ ]:
dbname = mongo_utils.get_database()

In [ ]:
urls = ['https://www.vesti.bg/rss.php',
    ]
feeds = [feedparser.parse(url)['entries'] for url in urls]
for url in urls:
    print(url)
#feeds[1][0].keys()
feed = [item for feed in feeds for item in feed]

In [ ]:
for newslink in feed:
  try:
    art_text = ''
    nlink=newslink["link"]
    collection_name = dbname["vesti_scrape_1"]
    if list(collection_name.find({"news_link" : nlink })):
        print("Article Exists")
    else:
        dt_obj=newslink["published"]
        date_posted = dt.strptime(dt_obj, '%a, %d %b %Y %H:%M:%S %z').timestamp()
        news_title=newslink["title"].replace('<p>','').replace('</p>','')
        page = requests.get(nlink)
        data = page.text
        soup = BeautifulSoup(data, 'html.parser')
        text_raw = soup.find('div', class_='article-text first-letter')
        for element in text_raw(text=lambda text: isinstance(text, Comment)):
              element.extract()
        text_lines=text_raw.findAll('p')
        for line in text_lines:
            plain_text=line.get_text()
            art_text += plain_text
        text = art_text.replace(' ','', 1).replace('\xa0','').replace('\n',' ').replace('\xa0', ' ').replace('\r', '').replace('\t', '')
        news_payload = {
        "date_posted" : date_posted,
        "news_link" : nlink,
        "news_text" : text,
        "news_title" : news_title
        }
        collection_name.insert_many([news_payload])
        print(news_payload)
  except:
    pass